In [5]:


import pandas as pd

#read the data
data=pd.read_csv('tcc_ceds_music.csv')


# data report with ydata profiling
from ydata_profiling import ProfileReport


#get the report

#profile = ProfileReport(data, title='Pandas Profiling Report', explorative=True)



/Users/brieucpopper/miniconda3/envs/tf/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#read the profile report
#profile.to_file("your_report.html")

Goals

-decision trees w/ pruning (use info gain)
-nn
-boosting ()
-SVM
-kNN


test on 2 classification problems

describe class problems, why they are interesting
non trivail but not too hard

testing and training rates (graphs)

analysis of results

max 12 pages


chat GPT summary of todo 
Why?

Understand the purpose of the project: Explore supervised learning techniques.
Realize that understanding algorithms requires testing them under various conditions.
Implement and compare simple learning algorithms.
Focus on analysis as the primary goal.
The Means

Choose datasets for exploration.
Tune the selected algorithms.
Write a comprehensive analysis.
Programming can be done in any language and libraries not specifically designed for this project.
Ensure your experiments can be recreated on a standard Linux machine.
Acceptable Libraries:

Machine learning algorithms: scikit-learn (Python), Weka (Java), e1071/nnet/random forest (R), ML toolbox (Matlab), tensorflow/pytorch (Python).
Scientific computing: numpy/scipy (Python), Matlab, R.
Plotting: matplotlib (Python), seaborn (Python), Matlab, R.
The Problems Given to You

Implement five learning algorithms:
Decision trees with pruning.
Neural networks.
Boosting.
Support Vector Machines with kernel function swapping.
k-nearest neighbors with different k values.
Testing

Design two classification problems.
Ensure they have non-trivial but clear characteristics for analysis.
What to Turn In

README.txt:
Instructions for running your code.
Provide URLs to access your code and data.
Include necessary files for reproducing your results.
yourgtaccount-analysis.pdf:
Describe your classification problems and why they are interesting.
Report training and testing error rates for each algorithm.
Include learning curves (performance vs. training size).
Analyze results, comparing and contrasting algorithms.
Suggest improvements for each algorithm.
Provide timing information (wall clock time and iterations).
Explain why you did not implement cross-validation if applicable.
Discuss the impact of data, hyperparameters, and other choices.
Determine which algorithm performed best and define "best."
Keep the analysis concise within a 12-page limit.
Scoring Criteria

Emphasize your analysis, as it carries the most weight.
Implementing code is not as important as understanding and explaining results.
Provide thorough but concise explanations.
Follow assignment format guidelines strictly.
Late submissions will not be accepted, so begin promptly.
Rescoring Criteria

Review feedback and request a rescore within a week.
Provide a clear explanation of where the grader made an error.
Failure to gain at least 5 points in a rescore request will result in losing 10 points for not internalizing feedback.
Plagiarism and Proper Citations

Avoid plagiarism; cite sources properly.
Do not use others' code for analysis, as it circumvents the learning process.
Engage in the process of exploring, tuning, and analyzing independently.





# Step 1 : understanding the dataset and pre-processing




28372 entries
release_date : int between 1950 and 2019, no missing values, Will Use
genre : string, can take 7 values, no missing values, will use
dating : from 0 to 0.64 
violence : 0 to 0.98
...

danceability : 0 to 0.99, normal distribution
loudness : kind of normal distribution
acousticness : 0 to 1, kind of inverse exponential distribution (lots of very low values)
instrumentalness : a lot of zeros, but still 0 to 1

valence : same, 0 to 1
energy : same, 0 to 1

topic : sadness, violence, world/life, obscene, music, night/time, romantic, feelings


release_date is highly overall correlated with loudness 
loudness is highly overall correlated with release_date 
acousticness is highly overall correlated with loudness 
energy is highly overall correlated with loudness 


---> won't use loudness 

## final columns :

release date ---> between -1 and 1 with 0

genre : one hot encoded for 7 values (pop, country, blues,jazz,reggae,rock,hip hop)

subjects : use them from 0 to 1 as they are (datin, violence, world, night, shake, family, romantic, comm, obscene, music, movement, light, family, like, sadness, feelings)

metrics on the song : use them as are (danceablity, acoustincess, intrismetnalness, valence, energy)




In [6]:
#Let's build X and y


import numpy as np
import sklearn
Xall=data.drop(['Unnamed: 0','artist_name','track_name','lyrics','len','loudness','topic','age'],axis=1)



#create new column encodedDate = (release date-1984.5)/34.5
# it is between -1 and 1

Xall['encodedDate']=(Xall['release_date']-1984.5)/34.5

Xall=Xall.drop(['release_date'],axis=1)

#one hot encode the genre
Xall=pd.get_dummies(Xall,columns=['genre'])

print(Xall.keys())




Index(['dating', 'violence', 'world/life', 'night/time', 'shake the audience',
       'family/gospel', 'romantic', 'communication', 'obscene', 'music',
       'movement/places', 'light/visual perceptions', 'family/spiritual',
       'like/girls', 'sadness', 'feelings', 'danceability', 'acousticness',
       'instrumentalness', 'valence', 'energy', 'encodedDate', 'genre_blues',
       'genre_country', 'genre_hip hop', 'genre_jazz', 'genre_pop',
       'genre_reggae', 'genre_rock'],
      dtype='object')


In [7]:
#first problem : predict if the date is before or after 1984.5 (is encodedDate is positive or not)

y=Xall['encodedDate']>0

X=Xall.drop(['encodedDate'],axis=1)

#use a decision tree classifier, get the accuracy

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

# #build the tree with gridsearch for hyperparams, and avoid overfitting
# from sklearn.model_selection import GridSearchCV
# param_grid={'max_depth':np.arange(2,10),'max_leaf_nodes':np.arange(6,12)}
# tree=GridSearchCV(DecisionTreeClassifier(),param_grid,cv=5)
# tree.fit(X_train,y_train)
# print(tree.best_params_)
# print(tree.score(X_test,y_test))

#we get 0.67 accuracy with max depth 4 and max leaf nodes 11










In [10]:
#now onto a simple ANN

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#build the model
model=keras.Sequential([
    layers.Dense(20,activation='relu',input_shape=[X.shape[1]]),
    layers.Dense(10,activation='relu'),
    layers.Dense(5,activation='relu'),
    layers.Dense(1)
])

#compile the model
model.compile(optimizer='adam',loss='mse',metrics=['mse','accuracy'])

#fit the model
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8)

Epoch 1/8
665/665 [==============================] - 5s 4ms/step - loss: 0.2025 - mse: 0.2025 - accuracy: 0.6891 - val_loss: 0.1829 - val_mse: 0.1829 - val_accuracy: 0.7292
Epoch 2/8
665/665 [==============================] - 2s 4ms/step - loss: 0.1799 - mse: 0.1799 - accuracy: 0.7285 - val_loss: 0.1794 - val_mse: 0.1794 - val_accuracy: 0.7285
Epoch 3/8
665/665 [==============================] - 2s 3ms/step - loss: 0.1774 - mse: 0.1774 - accuracy: 0.7356 - val_loss: 0.1792 - val_mse: 0.1792 - val_accuracy: 0.7296
Epoch 4/8
665/665 [==============================] - 3s 4ms/step - loss: 0.1765 - mse: 0.1765 - accuracy: 0.7358 - val_loss: 0.1779 - val_mse: 0.1779 - val_accuracy: 0.7293
Epoch 5/8
665/665 [==============================] - 3s 4ms/step - loss: 0.1748 - mse: 0.1748 - accuracy: 0.7361 - val_loss: 0.1785 - val_mse: 0.1785 - val_accuracy: 0.7318
Epoch 6/8
665/665 [==============================] - 3s 4ms/step - loss: 0.1745 - mse: 0.1745 - accuracy: 0.7363 - val_loss: 0.1772 - v

In [12]:
#get the accuracy on the test set
print(model.evaluate(X_test,y_test))


222/222 [==============================] - 1s 3ms/step - loss: 0.1768 - mse: 0.1768 - accuracy: 0.7326
[0.17675036191940308, 0.17675036191940308, 0.732553243637085]


In [18]:
#now with boosting trees wiht grid CV

from sklearn.ensemble import GradientBoostingClassifier

boost=GradientBoostingClassifier(n_estimators=300,learning_rate=0.1,max_depth=4,random_state=0)
boost.fit(X_train,y_train)

print(boost.score(X_test,y_test))



0.7403073452699845


In [19]:
#with SVM

from sklearn.svm import SVC

svm=SVC(kernel='rbf',C=1,gamma=1)
svm.fit(X_train,y_train)
print(svm.score(X_test,y_test))


0.7408712815451854


In [3]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
k = 75  
knn = KNeighborsClassifier(n_neighbors=k)

# Fit the classifier to the training data
knn.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 71.58%
